<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/lab_13_team_03/lab13_solution_team_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,700 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,452 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages 

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler


spark = SparkSession.builder.appName("WineClustering").getOrCreate()
df = spark.read.csv('/content/wine-clustering.csv', header=True, inferSchema=True)


In [3]:
feature_columns = df.columns
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(df).select("features")


In [6]:
k_values = [2, 10, 15, 20]
silhouette_scores = {}

for k in k_values:
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(data)
    predictions = model.transform(data)
    evaluator = ClusteringEvaluator()
    silhouette_score = evaluator.evaluate(predictions)
    silhouette_scores[k] = silhouette_score

    print(f"Score for k={k}: {silhouette_score}")


Score for k=2: 0.8213603513331723
Score for k=10: 0.6810238482437947
Score for k=15: 0.6844672306874956
Score for k=20: 0.6393344257672275


In [7]:
for k in k_values:
    print(f"Centers for k={k}:")
    for center in model.clusterCenters():
        print(center)


Centers for k=2:
[1.27141667e+01 3.29666667e+00 2.32916667e+00 2.17083333e+01
 9.53333333e+01 1.90416667e+00 1.22166667e+00 4.22500000e-01
 1.37333333e+00 4.72750000e+00 8.49166667e-01 2.23583333e+00
 5.65833333e+02]
[1.37166667e+01 1.81333333e+00 2.59666667e+00 1.67666667e+01
 1.05333333e+02 2.81666667e+00 2.82000000e+00 3.26666667e-01
 1.65000000e+00 6.40000000e+00 1.18666667e+00 2.90333333e+00
 1.33500000e+03]
[1.32000e+01 2.87375e+00 2.46000e+00 1.94875e+01 1.13375e+02 2.14125e+00
 1.78875e+00 3.67500e-01 1.60375e+00 6.25500e+00 8.25000e-01 2.57750e+00
 8.34375e+02]
[ 12.29    2.004   2.12   19.8    88.8     2.292   2.132   0.372   1.628
   2.41    1.03    3.104 304.   ]
[1.3452e+01 1.6800e+00 2.3160e+00 1.9700e+01 1.2480e+02 2.9040e+00
 2.8480e+00 2.8400e-01 2.2760e+00 4.4540e+00 1.1100e+00 3.1280e+00
 9.7340e+02]
[1.36514286e+01 1.80571429e+00 2.46000000e+00 1.61428571e+01
 1.02714286e+02 3.03714286e+00 3.29428571e+00 3.08571429e-01
 2.28857143e+00 6.14000000e+00 1.06285714e+00 2